In [5]:
conda install numpy=1.25

Channels:
 - defaults
Platform: win-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import json

from openaq import OpenAQ
import httpx

import geopandas as gpd

import pprint

import shapely
from shapely.geometry import shape
from shapely.geometry import Point

import rioxarray as rxr

import rasterio as rio
from rasterio.features import shapes

In [ ]:
client = OpenAQ(api_key='0___0')

In [ ]:
# Load GeoJSON file
los_angeles = gpd.read_file("../Data/Inputs/Boundary_Shapefile/City_Boundaries.geojson")

In [ ]:
los_angeles.crs

In [ ]:
print(los_angeles.head())

In [ ]:
los_angeles.plot();

In [ ]:
type(los_angeles)

In [ ]:
bbox1 = los_angeles.total_bounds

In [ ]:
bbox1

In [ ]:
bbox_str = ",".join(map(str, bbox1))
bbox_str

The OpenAI documentation notes that the CRS for the station data is WGS84:

https://docs.openaq.org/resources/locations

As a reminder, this is also the same CRS as the LA basemap we are using to query the data:

In [ ]:
los_angeles.crs.name

locations = client.locations.list(
    bbox = ",".join(map(str, los_angeles.total_bounds)), limit=1000
)
#pprint.pp(locations)
client.close()

In [ ]:
type(locations)

In [ ]:
dir(locations)

In [ ]:
locations.results[0]

In [ ]:
results = locations.results

In [ ]:
len(results)

In [ ]:
locations.headers

In [ ]:
locations.meta

In [ ]:
locations.results[0]

In [ ]:
help(locations.json)

locations.json()

In [ ]:
locations_json = json.loads(locations.json())

In [ ]:
type(locations_json)

locations_json

In [ ]:
results = locations_json['results']

In [ ]:
la_df = pd.json_normalize(results)

In [ ]:
la_df['geometry'] = la_df.apply(lambda row: Point(row['coordinates.longitude'], row['coordinates.latitude']), axis=1)
la_gdf = gpd.GeoDataFrame(la_df, geometry='geometry', crs="EPSG:4326")

In [ ]:
la_gdf.head()

The nested data still looks good

In [ ]:
la_gdf['sensors'][0]

In [ ]:
la_gdf['instruments'][0]

In [ ]:
la_gdf[la_gdf['id']==1036]

In [ ]:
la_gdf.sort_values('id')

In [ ]:
la_gdf.columns

In [ ]:
la_gdf.set_index('id',inplace=True)

In [ ]:
la_gdf.head()

There seems to be two columns `datetimeFirst` and `datetimeLast` which could be nulls

In [ ]:
la_gdf['datetimeFirst'].unique()

In [ ]:
la_gdf['datetimeLast'].unique()

In [ ]:
la_gdf = la_gdf.drop(['datetimeFirst','datetimeLast'],axis=1)

In [ ]:
la_gdf.head()

In [ ]:
la_gdf.columns

The `datetimeFirst.utc`, `datetimeLast.utc` and `timezone` columns are also going to be redundant - the entire dataset is in the same timezone.

In [ ]:
la_gdf = la_gdf.drop(['datetimeFirst.utc', 'datetimeLast.utc', 'timezone'],axis=1)

In [ ]:
la_gdf.head()

In [ ]:
la_gdf['sensors'][847]

In [ ]:
la_gdf['sensors'][847][0]['name']

I have to look into this more, but I think the 'id' for the sensor is different than the station id, i.e. it is safe to extract this sensor data and map it to the overall station id.

In [ ]:
la_gdf['sensors'][847][0]['parameter']['name']

In [ ]:
la_gdf['sensors']

In [ ]:
la_dgf['sensors'][1052][0]['name']

In [ ]:
pollutant = [la_gdf['sensors'][i][0]['parameter']['name'] for i in la_df.index]

The sensor types for the first 5 stations (sorted by station index asc)

In [ ]:
pollutant[:5]

In [ ]:
la_gdf.insert(loc = 1, column = 'pollutant', value = pollutant)

Now we have an easy to read `pollutant` column indicating what the station monitors. Each station only monitors one type of pollutant.

In [ ]:
la_gdf.head()

In [ ]:
la_gdf['pollutant'].isna().sum()

The vast majority of these stations are monitoring pm1. For a robust map, this would likely be the best option, but it would be good to check spatial distribution.

In [ ]:
la_gdf['pollutant'].value_counts(normalize=True)*100

In [ ]:
la_gdf[la_gdf['pollutant'] == 'pm1'].head()

Next step is use Folium to visualize distribution of these stations

In [ ]:
type(la_gdf)

In [ ]:
la_gdf.head()

In [ ]:
la_gdf.to_file("../Data/Outputs/stations.geojson", driver="GeoJSON")

### Clipping stations to LA outline

In [ ]:
la_gdf = gpd.read_file("../Data/Outputs/stations.geojson")

In [ ]:
la_dem = rxr.open_rasterio(r'../Data/Inputs/DEMs/Outputs/la_dem.tif', masked=True)

stations_clip = la_dem.rio.clip(la_basemap.geometry, la_basemap.crs, drop=True, invert=False)

gpd.clip(la_gdf, la_dem)